<a href="https://colab.research.google.com/github/Ritesh-Sil/Santander-Customer-Transaction-Prediction-Using-GBDT-and-Random-Forest/blob/Master/GBDT_Santander_Customer_Transaction_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparing the Kaggle Stuffs

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list -s 'Santander Customer Transaction Prediction'

ref                                                                title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
lakshmi25npathi/santander-customer-transaction-prediction-dataset  Santander Customer Transaction Prediction Dataset  250MB  2019-05-24 06:17:57           1660         33  0.5882353        
mrushan3/santander-customer-transaction-prediction                 santander customer transaction prediction            8MB  2020-01-30 04:52:15             76          7  0.23529412       
thaikeras/the-shakeups-of-10-competitions                          The Shakeups of 10 Competitions                      3MB  2019-04-17 14:10:17             32          1  0.0625           


In [ ]:
!mkdir /content/santander

In [ ]:
!cd /content/santander

In [ ]:
!kaggle datasets download lakshmi25npathi/santander-customer-transaction-prediction-dataset

 95% 237M/250M [00:03<00:00, 74.0MB/s]
100% 250M/250M [00:03<00:00, 84.8MB/s]


In [ ]:
!pwd

/content


In [ ]:
!cd /content/santander/

In [ ]:
!unzip /content/santander/santander-customer-transaction-prediction-dataset.zip -d santander

Archive:  /content/santander/santander-customer-transaction-prediction-dataset.zip
  inflating: santander/test.csv      
  inflating: santander/train.csv     


In [ ]:
!rm -rf /content/santander/santander-customer-transaction-prediction-dataset.zip

## Starting off to actual work

**Importing the libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

In [ ]:
df_train  = pd.read_csv('/content/santander/train.csv')
df_train.sample(5)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
6607,train_6607,0,14.8553,3.1409,8.4104,9.4878,6.7708,-4.5427,4.9391,14.6826,...,-2.8402,9.2288,0.0013,-3.3478,17.5555,-1.0750,5.8528,9.0395,21.2842,11.5364
41246,train_41246,1,10.5040,3.1098,12.9729,8.2943,10.6950,-0.4540,5.9444,13.4361,...,4.0623,2.6515,1.2439,1.9726,23.2947,-0.0526,-3.7539,8.4528,8.7480,5.2857
109218,train_109218,0,14.3152,-0.6827,13.2676,5.1086,11.2682,4.3155,6.3581,20.3985,...,5.7971,7.9609,1.9052,10.8890,13.7267,1.3525,-1.7636,8.3139,14.2670,4.0273
58588,train_58588,0,13.9320,-9.4467,8.0982,8.8652,8.8455,-17.0120,5.4911,18.8186,...,7.8636,6.3159,5.7037,8.5358,13.1309,1.9117,12.6298,9.7809,11.9195,1.4865
6445,train_6445,0,10.6114,-2.7679,7.9435,3.3475,11.9259,-0.2253,5.7975,11.2915,...,6.4769,6.2562,1.8225,-6.4596,18.7924,-0.3097,-1.2372,8.6005,15.9695,-7.8367


In [ ]:
df_train = df_train.drop(['ID_code'], axis=1)

In [ ]:
X = df_train.drop(['target'], axis=1)
y = df_train['target']

In [ ]:
X.sample(5)

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
55660,13.8779,-1.1218,13.4303,7.4022,10.8302,0.2110,5.3208,17.0175,6.4567,8.4591,...,6.0713,9.2721,-0.3187,9.2693,18.1709,0.2039,1.8633,7.9049,17.3561,3.2989
81723,12.5368,-2.9250,12.9431,6.2285,14.1705,6.8867,5.0953,11.1952,1.0588,8.9841,...,3.6045,3.8091,1.9575,1.5772,14.7957,1.4413,3.6802,8.0102,14.1002,3.1665
131445,12.7193,-2.9061,10.5189,5.1946,12.8099,-18.5732,5.2670,15.7204,5.4624,6.3062,...,-1.9219,5.5784,1.6070,8.8838,19.1044,-0.1528,12.4607,9.6928,15.9378,-0.6059
47800,8.0423,2.3686,9.0603,7.2908,9.3633,2.3617,4.6056,16.0864,-1.2803,7.6846,...,0.1395,8.7650,2.0233,4.6744,13.1866,-1.2879,1.8988,9.1004,17.2588,-9.7761
105035,17.7174,-6.2069,11.3634,5.9928,13.3340,7.1912,5.0301,14.1267,-4.9890,9.0275,...,1.7732,4.8350,2.1923,10.3905,19.8791,-2.1644,10.4597,7.4648,18.0987,-9.5233


In [ ]:
y.sample(5)

117641    0
70551     0
168468    0
180876    1
143542    0
Name: target, dtype: int64

**Splitting the data into train and test dataset**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

**Machine Learning using - Gradient Boosting Classifier**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

The three main hyperparameters for the GBDT classifier model is 


1.   n_estimators = Which are nothing but the base learners
2.   learning rate = Shrinkgae term
3.   max_depth = It is coming from the decission tree concept

In [ ]:
gbc = GradientBoostingClassifier(n_estimators = 5000, #Nummber of base learners
                                 learning_rate=0.05, # Learning rate for pseudo residuals
                                 max_depth=3,  #Max_depth could be choosen less, to make the base learners high bias
                                 subsample = 0.5, 
                                 max_features='log2', 
                                 verbose = 1, 
                                 random_state=42,
                                 n_iter_no_change=20)

In [ ]:
%%time
gbc.fit(X_train,y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.6451           0.0014           19.65m
         2           0.6468           0.0011           19.78m
         3           0.6473           0.0012           20.11m
         4           0.6492           0.0007           20.20m
         5           0.6422           0.0007           20.14m
         6           0.6466           0.0014           20.18m
         7           0.6392           0.0010           20.24m
         8           0.6403           0.0011           20.17m
         9           0.6419           0.0012           20.16m
        10           0.6450           0.0009           20.17m
        20           0.6262           0.0008           20.11m
        30           0.6184           0.0010           19.98m
        40           0.6056           0.0008           19.93m
        50           0.6008           0.0007           19.94m
        60           0.5953           0.0006           19.92m
       

GradientBoostingClassifier(learning_rate=0.05, max_features='log2',
                           n_estimators=5000, n_iter_no_change=20,
                           random_state=42, subsample=0.5, verbose=1)

We can store the model into .pkl format for future use

In [ ]:
import joblib
joblib.dump(gbc,'gbc.pkl')

['gbc.pkl']

In [ ]:
GBDT_clf = joblib.load('/content/gbc.pkl')

In [ ]:
# Probability prediction for class label = 1
GBDT_clf.predict_proba(X_test).T[1]

array([0.00173276, 0.04412305, 0.01148146, ..., 0.07995443, 0.27642039,
       0.02276288])